## Overview
The method I intended to use and title 'Sample' starts by giving
ChatGPT the code previously created that is a Python-based image annotation 
code, but does not meet the specifications required. Note that Sample 1 and Sample 2 vary.
Using a sample code as a baseline, I then used ChatGPT 
to further develop the code.
The general process is to suggest a new feature, debug,
suggest a new feature, debug, etc.
The goal is to eventually evolve the sample program into 
something that meets the needs of the research.

In [ ]:
# Sample 1:

from tkinter import *
from tkinter.colorchooser import askcolor
from tkinter import filedialog
from PIL import ImageTk, Image, ImageGrab


class Image_Markup(object):

    DEFAULT_COLOR = 'black'
    Image_Width = 900
    Image_Height = 600


    def __init__(self):
        self.root = Tk()
        self.root.title('SEE Image Markup Tool')

        self.point_button = Button(self.root, text='Point Select', command=self.use_point_select)
        self.point_button.grid(row=0, column=0)

        self.brush_button = Button(self.root, text='Brush', command=self.use_brush)
        self.brush_button.grid(row=0, column=1)

        self.color_button = Button(self.root, text='Color', command=self.choose_color)
        self.color_button.grid(row=0, column=2)

        self.eraser_button = Button(self.root, text='Eraser', command=self.use_eraser)
        self.eraser_button.grid(row=0, column=3)

        self.browse_button = Button(self.root, text='Browse', command=self.browseFiles)
        self.browse_button.grid(row=0, column=4)

        self.clear_button = Button(self.root, text='Clear All', command=self.clear_all)
        self.clear_button.grid(row=0, column=5)

        self.choose_size_button = Scale(self.root, from_=10, to=50, orient=HORIZONTAL)
        self.choose_size_button.grid(row=0, column=6)

        self.save_button = Button(self.root, text='Save', command=self.save_canvas)
        self.save_button.grid(row=0, column=9)

        self.c = Canvas(self.root, bg='white', width=self.Image_Width, height=self.Image_Height)
        self.c.grid(row=1, columnspan=10)

        self.setup()
        self.root.mainloop()

    def setup(self):
        self.old_x = None
        self.old_y = None
        self.line_width = self.choose_size_button.get()
        self.color = self.DEFAULT_COLOR
        self.eraser_on = False
        self.active_button = self.brush_button
        self.c.bind('<B1-Motion>', self.paint)
        self.c.bind('<ButtonRelease-1>', self.reset)

    def use_point_select(self):
        self.activate_button(self.brush_button)

    def use_brush(self):
        self.activate_button(self.brush_button)

    def choose_color(self):
        self.eraser_on = False
        self.color = askcolor(color=self.color)[1]

    def use_eraser(self):
        self.activate_button(self.eraser_button, eraser_mode=True)

    def clear_all(self):
        self.c.delete("all")

    def activate_button(self, some_button, eraser_mode=False):
        self.active_button.config(relief=RAISED)
        some_button.config(relief=SUNKEN)
        self.active_button = some_button
        self.eraser_on = eraser_mode

    def paint(self, event):
        self.line_width = self.choose_size_button.get()
        paint_color = 'white' if self.eraser_on else self.color
        if self.old_x and self.old_y:
            self.c.create_line(self.old_x, self.old_y, event.x, event.y,
                               width=self.line_width, fill=paint_color,
                               capstyle=ROUND, smooth=TRUE, splinesteps=36)
        self.old_x = event.x
        self.old_y = event.y

    def reset(self, event):
        self.old_x, self.old_y = None, None

    def save_canvas(self):
        x = self.root.winfo_rootx() + self.c.winfo_x()
        y = self.root.winfo_rooty() + self.c.winfo_y()
        x1 = x + self.c.winfo_width()
        y1 = y + self.c.winfo_height()
        print('x, y, x1, y1:', x,y,x1,y1)
        ImageGrab.grab().crop((self.c.bbox())).save("/Users/ishaan/Desktop/SavedCanvases/test.png")

    def browseFiles(self):
        filename = filedialog.askopenfilename(initialdir = "/",
                                            title = "Select a File",
                                            filetypes = (("png files", "*.png"),
                                                        ("jpg files", "*.jpg"),
                                                        ("jpeg files", "*.jpeg")))
        
        img = ImageTk.PhotoImage(Image.open(filename))
        self.Image_Width = img.width()
        self.Image_Height = img.height()
        self.c.create_image(10,10, anchor=NW,image=img)
        self.root.mainloop()



if __name__ == '__main__':
    Image_Markup()

# From git repository: 
# see-insight/see-tkinter-gui/GUI-Basic.py

In [ ]:
# Sample 2:

import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageDraw, ImageTk
import time

class ImageSegmentationGUI:
    def __init__(self, root):
        self.root = root
        self.canvas = None
        self.image = None
        self.segmented_image = None
        self.brush_size = 1
        self.zoom_level = 1.0
        self.brush_color = (255, 0, 0)  # Red color

        self.create_menu()
        self.create_canvas()

    def create_menu(self):
        menu_bar = tk.Menu(self.root)
        file_menu = tk.Menu(menu_bar, tearoff=0)
        file_menu.add_command(label="Open Image", command=self.open_image)
        file_menu.add_command(label="Export Segmentation", command=self.export_segmentation)
        menu_bar.add_cascade(label="File", menu=file_menu)
        settings_menu = tk.Menu(menu_bar, tearoff=0)
        settings_menu.add_command(label="Zoom In", command=self.zoom_in)
        settings_menu.add_command(label="Zoom Out", command=self.zoom_out)
        menu_bar.add_cascade(label="Settings", menu=settings_menu)
        self.root.config(menu=menu_bar)

    def create_canvas(self):
        self.canvas = tk.Canvas(self.root, bg="white")
        self.canvas.pack(fill=tk.BOTH, expand=True)
        self.canvas.bind("<B1-Motion>", self.segment_image)
        self.canvas.bind("<Button-2>", self.fill_segmentation)

    def update_canvas(self):
            if self.image:
                width = int(self.image.width * self.zoom_level)
                height = int(self.image.height * self.zoom_level)
                resized_image = self.image.resize((width, height), Image.ANTIALIAS)
                self.image_tk = ImageTk.PhotoImage(resized_image)
                self.canvas.config(width=width, height=height)
                self.canvas.create_image(0, 0, anchor=tk.NW, image=self.image_tk)


    def zoom_in(self):
        self.zoom_level *= 1.2
        self.update_canvas()

    def zoom_out(self):
        self.zoom_level /= 1.2
        self.update_canvas()

    def segment_image(self, event):
        if self.image:
            draw = ImageDraw.Draw(self.segmented_image)
            x1 = int(event.x / self.zoom_level) - self.brush_size
            y1 = int(event.y / self.zoom_level) - self.brush_size
            x2 = int(event.x / self.zoom_level) + self.brush_size
            y2 = int(event.y / self.zoom_level) + self.brush_size
            draw.ellipse([x1, y1, x2, y2], fill=self.brush_color)
            self.canvas.create_oval(x1 * self.zoom_level, y1 * self.zoom_level,
                                    x2 * self.zoom_level, y2 * self.zoom_level, fill='red')



    def open_image(self):
        image_path = filedialog.askopenfilename()#filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        if image_path:
            self.image = Image.open(image_path)
            self.segmented_image = self.image.copy()
            self.image_tk = ImageTk.PhotoImage(self.image)  # Convert PIL Image to Tkinter PhotoImage
            self.canvas.config(width=self.image.width, height=self.image.height)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.image_tk)

    
    def export_segmentation(self):
        if self.segmented_image:
            save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG", "*.png")])
            if save_path:
                width, height = self.segmented_image.size
                segmented_pattern = Image.new("L", (width, height), color=0)  # Create a new black and white image
                draw = ImageDraw.Draw(segmented_pattern)
                for y in range(height):
                    for x in range(width):
                        r, g, b = self.segmented_image.getpixel((x, y))
                        if r != 255 or g != 0 or b != 0:
                            draw.point((x, y), fill=255)  # Set non-red pixels to white
                segmented_pattern.save(save_path)

    def fill_segmentation(self, event):
        if self.image:
            seed_x = int(event.x / self.zoom_level)
            seed_y = int(event.y / self.zoom_level)
            seed_color = self.image.getpixel((seed_x, seed_y))  # Get the color from the original image
            if seed_color != self.brush_color:
                self.flood_fill(seed_x, seed_y, seed_color)

    def flood_fill(self, x, y, seed_color):
        width, height = self.image.size  # Use the original image size
        stack = [(x, y)]

        while stack:
            curr_x, curr_y = stack.pop()
            if 0 <= curr_x < width and 0 <= curr_y < height:
                curr_color = self.image.getpixel((curr_x, curr_y))  # Get the color from the original image
                if curr_color == seed_color:
                    self.segmented_image.putpixel((curr_x, curr_y), self.brush_color)

                    #stack.append((curr_x - 1, curr_y))
                    stack.append((curr_x + 1, curr_y))
                    #stack.append((curr_x, curr_y - 1))
                    stack.append((curr_x, curr_y + 1))

        #self.update_canvas()



if __name__ == "__main__":

    root = tk.Tk()
    root.title("Image Segmentation")
    app = ImageSegmentationGUI(root)
    root.mainloop()

# From Nathan